In [1]:
# General python imports
import numpy as np
import pandas as pd
import warnings

# Updadte this option setting as you prefer
pd.set_option('display.max_rows', 5)

# Astropy
from astropy.visualization import  ZScaleInterval, AsinhStretch
from astropy import units as u
from astropy.coordinates import SkyCoord

# Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query


In [2]:
# Get a Rubin TAP service instance. 
service = get_tap_service()
assert service is not None

In [3]:
# Define a reference position on the sky and a radius in arcseconds for a cone search
# TODOO Change the sky position to have SNe 
c1 = SkyCoord(ra=59.7955707*u.degree, dec=-29.91176471*u.degree, frame='icrs')
radius = 5 * u.arcmin

In [4]:
query = "SELECT obj.ra, obj.dec, obj.objectId, obj.mag_g, obj.mag_r, " \
        "obj.mag_i, obj.mag_g_cModel, obj.mag_r_cModel, obj.mag_i_cModel," \
        "obj.psFlux_g, obj.psFlux_r, obj.psFlux_i, obj.cModelFlux_g, " \
        "obj.cModelFlux_r, obj.cModelFlux_i, obj.tract, obj.patch, " \
        "obj.extendedness, obj.good, obj.clean, " \
        "truth.mag_r as truth_mag_r, truth.match_objectId, "\
        "truth.flux_g, truth.flux_r, truth.flux_i, truth.truth_type, " \
        "truth.match_sep, truth.is_variable " \
        "FROM dp01_dc2_catalogs.object as obj " \
        "JOIN dp01_dc2_catalogs.truth_match as truth " \
        "ON truth.match_objectId = obj.objectId " \
        "WHERE CONTAINS(POINT('ICRS', obj.ra, obj.dec),"\
        "CIRCLE('ICRS', " + str(c1.ra.value) + ", " + str(c1.dec.value) + ", " \
        + str(radius.to(u.deg).value) + " )) = 1 " \
        "AND truth.match_objectid >= 0 "\
        "AND truth.is_good_match = 1"

In [7]:
%%time
# Execute the query and convert the results to a pandas dataframe
data = service.search(query).to_table().to_pandas() 
assert len(data) == 10284

CPU times: user 738 ms, sys: 19.7 ms, total: 758 ms
Wall time: 1.48 s


In [ ]:
# Map the truth type to a descriptive string
data['truth_type'] = data['truth_type'].map(
    {1: 'galaxy', 2: 'star', 3: 'SNe'})

In [21]:
data['psFlux_g'].to_numpy()

array([127.3305653,  78.2418751,  59.2568471, ...,  57.5786937,
       239.210167 ,  34.5541725])

In [25]:
# To convert between nJy and AB magnitudes (or other flux units), we recommend using astropy.units. 
# For example, you can see that 575 nJy is approximately the 5-sigma 30s imaging depth of LSST, and the AB magnitude zero is nearly,
(575*u.nJy).to(u.ABmag)

<Magnitude 24.50083039 mag(AB)>

In [26]:
# but not exactly, 3631 Jy as it is often hard coded to be:
zero_Nj = (0*u.ABmag).to_value(u.nJy)
zero_Nj

3630780547701.003

In [23]:
# Convert flux to magnitude. psfFlux has the localPhotoCalib already applied? Just use astropy to convert to magnitude
data['psfMag_g'] = (data['psFlux_g'].to_numpy()*u.nJy).to(u.ABmag)

0        26.137668
1        26.666402
           ...    
10282    25.453051
10283    27.553749
Name: psfMag_g, Length: 10284, dtype: float64

In [24]:
data['psfMag_g']

0        26.137668
1        26.666402
           ...    
10282    25.453051
10283    27.553749
Name: psfMag_g, Length: 10284, dtype: float64

In [ ]:
# instFlux columns are in counts. These columns are only in the reference and forced_photometry catalogs. To convert to flux: 
# https://community.lsst.org/t/photocalib-has-replaced-calib-welcoming-our-nanojansky-overlords/3648
flux = instFlux*localPhotoCalib